In [21]:
# TODO : 경로를 확인하여 주세요.
%run ./Dataset.ipynb

Dataset 클래스 호출


In [22]:
# Office_Image_Dataset 클래스 
# 목적 : Dataset 클래스를 상속받습니다. 
class Office_Image_Dataset(Dataset):
    print("Office_Image_Dataset(Dataset) 클래스 호출")
    # 코드 확장성을 위해 데코레이터를 통한 base 메서드를 정의합니다. 
    # 이는 Dataset 클래스 내의 메서드를 더 쉽게 호출하거나 오버라이딩이 다음과 같이 가능해집니다. 
    # self.base.forward...()
    @property
    # base() 메서드를 통해 
    def base(self):
        return super(Office_Image_Dataset, self)

Office_Image_Dataset(Dataset) 클래스 호출


In [24]:
# Office_Image_Dataset.__init__ 의 역할 
# 목적 : 이미지 
# resolution : 이미지 가로 세로 크기
# input_shape : 데이터의 마지막 차원값을 호출
def office_image_init(self, resolution = [100, 100], input_shape = [-1]):
    print("Office_Image_Dataset.__init__ 호출")
    
    # Dataset 클래스의 __init__ 메소드를 호출하여 인스턴스 초기화 과정
    self.base.__init__(dataset_name = 'office_image_dataset', ai_mode = 'dual_select')

    # 데이터 저장 경로 변수화
    # TODO : All_Image 폴더를 기준으로 상위 폴더명을 입력합니다. 
    path = './Data/All_Image_mini'
    
    # 폴더명이 종속변수 이기에 폴더명을 변수화 합니다.
    # office, smart ,webcam
    domain_names = list_dir(path)    

    # 이미지 픽셀정보, 데이터 인덱싱, 오브젝트 정보를 담기위한 빈 리스트 생성 
    images = []
    didxs, oidxs = [], []

    # dx : 도메인 인덱싱 (0,1,2)
    # dname : 도메인 명 (office, smart ,webcam)
    for dx, dname in enumerate(domain_names):
        
        # All_Image 의 하위 폴더까지의 경로를 변수처리합니다. 
        # domainpath :  .../All_Image/office
        # domainpath :  .../All_Image/smart
        # domainpath :  .../All_Image/webcam
        domainpath = os.path.join(path, dname)

        # 상위 도메인 경로의 하위 폴더명을 변수화 합니다. 
        # back_pack, bike, ...
        object_names = list_dir(domainpath)
        
        # 오브젝트 단위의 반복문을 수행합니다. 
        # ox : 오브젝트 단위 인덱스 
        # oname : 오브젝트 이름 
        for ox, oname in enumerate(object_names):
            
            # 오브젝트 기준 폴더명 까지 경로화 합니다. 
            objectpath = os.path.join(domainpath, oname)
            
            # 오브젝트 경로 하위 이미지 파일 이름을 변수화 합니다. 
            filenames = list_dir(objectpath)

            # 이미지 파일의 경우 .jpg 가 아닌 경우 예외처리 로직이 존재합니다. 
            for fname in filenames:
            
                if fname[-4:] != '.jpg':
                    continue
                
                # 파일 이미지 경로 변수화 
                imagepath = os.path.join(objectpath, fname)
                # 이미지에 대한 1차원 픽셀화 
                pixels = load_image_pixels(imagepath=imagepath, resolution=resolution, input_shape=input_shape)
                
                # 이미지 데이터의 픽셀화 이후 저장
                images.append(pixels)
                
                # 도메인 인덱싱 정보 
                didxs.append(dx)
                # 오브젝트 인덱싱 정보 
                oidxs.append(ox)

    # 반복문 탈출
    # 제공되는 데이터는 컬러 3 차원 데이터 이므로 [100,100,3]
    self.image_shape = resolution + [3]

    # 독립변수 형 일원화
    # images 변수는 np.asarray() 를 통해 형 변환 
    xs = np.asarray(images, np.float32)

    # 두 가지의 종속변수 생성 
    # ys0(도메인), ys1(오브젝트)
    ys0 = onehot(didxs, len(domain_names))
    ys1 = onehot(oidxs, len(object_names))

    # 현재 종속변수는 두 개로 나뉘어지나, 수평 형식의 하나의 리스트로 형 변환을 수행 
    ys = np.hstack([ys0, ys1])

    # 생성된 독립변수와 종속변수를 기반으로 데이터 셔플링 후 학습, 검증, 테스트 데이터 생성 
    self.shuffle_data(xs, ys, 0.8)
    
    # 두 도메인에 대하여 종속변수처리
    self.target_names = [domain_names, object_names]
    # [['office', 'smart', 'webcam'], ['back_pack', 'bike', 'bike_helmet', ...]]
    
    # 도메인과 오브젝트 구분을 위하여 도메인의 수량 파악
    self.cnts = [len(domain_names)]
    # self.cnts = [3]
    
Office_Image_Dataset.__init__ = office_image_init

In [25]:

# Office_Image_Dataset.forward_postproc 의 역할 
# 목적 : 손실 연산 
def office_image_forward_postproc(self, output, y):
    print("Office_Image_Dataset.forward_postproc 호출")
    
    # 전체 예측 결괏값인 output 을 self.cnts 를 기준으로
    # 도메인과 오브젝트로 구분하여 줍니다. 
    outputs = np.hsplit(output, self.cnts)

    # 실제 결괏값 또한 self.cnts 를 기준으로 
    # 도메인과 오브젝트로 구분하여 줍니다. 
    ys = np.hsplit(y, self.cnts)

    # 도메인 예측값 outputs[0], 도메인 실제값 ys[0], 도메인 손실값 loss0
    # 오브젝트 예측값 outputs[1], 오브젝트 실제값 ys[1], 오브젝트 손실값 loss1
    loss0, aux0 = self.base.forward_postproc(outputs[0], ys[0], 'select')
    loss1, aux1 = self.base.forward_postproc(outputs[1], ys[1], 'select')
    
    # 손실 정보는 합산하여 처리합니다. 
    return loss0 + loss1, [aux0, aux1]

Office_Image_Dataset.forward_postproc = office_image_forward_postproc

In [7]:
# Office_Image_Dataset.backprop_postproc 의 역할 
# 목표 : G_output 연산 
def office_image_backprop_postproc(self, G_loss, aux):
    print("Office_Image_Dataset.backprop_postproc 호출")
    aux0, aux1 = aux

    # 동일한 기울기에서 시작합니다. 
    G_output0 = self.base.backprop_postproc(G_loss=G_loss, aux = aux0, ai_mode = 'select')
    G_output1 = self.base.backprop_postproc(G_loss=G_loss, aux = aux1, ai_mode = 'select')

    # np.hstack 를 활용해 하나의 G_output 으로 처리 및 반환 
    return np.hstack([G_output0, G_output1])

Office_Image_Dataset.backprop_postproc = office_image_backprop_postproc

In [8]:
# Office_Image_Dataset.eval_accuracy 의 역할 
# 목표 : 오브젝트와 도메인에 대하여 정확도 연산 
# x : 독립변수
# y : 종속변수 
# output : 예측
def office_image_eval_accuracy(self, x, y, output):
    print("Office_Image_Dataset.eval_accuracy 호출")
    
    # 도메인과 오브젝트로 나눠 예측하기 위한 분리 과정
    outputs = np.hsplit(output,self.cnts)
    ys = np.hsplit(y, self.cnts)
    
    # 독립변수를 활용해 도메인과 오브젝트로 나눠 정확도를 산출합니다. 
    acc0 = self.base.eval_accuracy(x, ys[0], outputs[0], 'select')
    acc1 = self.base.eval_accuracy(x, ys[1], outputs[1], 'select')

    # 각 산출된 정확도는 리스트로 묶어 반환처리 합니다. 
    # 추후 인덱싱을 통해 값을 외부로 출력합니다. 
    return [acc0, acc1]

Office_Image_Dataset.eval_accuracy = office_image_eval_accuracy

In [9]:
# Office_Image_Dataset.train_prt_result 의 역할 
# 목적 : 
# 테스트 결과를 사용자에게 출력하기 위한 메서드
# 매개변수는 에폭, 학습 손실평균, 학습 정확도평균, 검증 정확도, 주기별 소요시간, 누적 소요시간 입니다. 
def office_image_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    print("Office_Image_Dataset.train_prt_result 호출")
    
    # 테스트 결과에 따른 
    acc_pair = np.mean(accs, axis = 0)

    # 결과 출력 기능
    print('[Epoch {}] (Tr)Cost={:5.3f},'\
          '(Tr)Domain_acc:{:5.3f} | (Tr)Object_acc:{:5.3f} | (V)Domain_acc:{:5.3f} | (V)Object_acc:{:5.3f} ({}/{} secs)'.\
            format(epoch, np.mean(costs), 
                   acc_pair[0], acc_pair[1], 
                   acc[0], acc[1], 
                   time1, time2))
    
Office_Image_Dataset.train_prt_result = office_image_train_prt_result

In [10]:
# Office_Image_Dataset.test_prt_result 의 역할 
# 목적 : 테스트 결과의 출력
def adam_test_prt_result(self, name, acc, time):
    print("Office_Image_Dataset.test_prt_result 호출")
    
    print('[안내] {} AI Model Report : (Te)Domain_acc:{:5.3f} | (Te)Object_acc:{:5.3f} ({} secs)\n'.\
          format(name, acc[0], acc[1], time))
    
Office_Image_Dataset.test_prt_result = adam_test_prt_result

In [11]:
# Office_Image_Dataset.get_estimate 의 역할
# 목적 : 도메인과 오브젝트를 구분하여 예측 결과를 반환 
def office_image_get_estimate(self, output):
    print("Office_Image_Dataset.get_estimate 호출")
    outputs = np.hsplit(output, self.cnts)

    estimate0 = self.base.get_estimate(outputs[0], 'select')
    estimate1 = self.base.get_estimate(outputs[1], 'select')
    
    return np.hstack([estimate0, estimate1])

Office_Image_Dataset.get_estimate = office_image_get_estimate

In [12]:
# Office_Image_Dataset.visualize 의 역할 
# 목적 : 예측 결과에 따른 시각화 
def office_image_visualize(self, xs, estimates, answers):
    print("Office_Image_Dataset.visualize 호출")
    
    draw_images_horz(xs, self.image_shape)

    ests, anss = np.hsplit(estimates, self.cnts), np.hsplit(answers, self.cnts)

    captions = ['도메인', '상품(오브젝트)']

    for m in range(2):
        print('[{} 추정결과]'.format(captions[m]))
        show_select_results(ests[m], anss[m], self.target_names[m], 8)

Office_Image_Dataset.visualize = office_image_visualize